In [ ]:
import pandas as pd
import numpy as np
from autocorrect import spell # autocorrect information
import spacy # might useless - but is a useful package to implement < - 
from tqdm.auto import tqdm
import helper
from helper import *
import plotly.graph_objects as go
import plotly.express as px
#tqdm.pandas()

In [ ]:
input_survey=pd.read_csv('../data/WebsiteSatisfaction.csv')
#start to rename the csv hard coding - throw exception if the required schema in the csv doesnt exist
#drop the na and tell the business to get fucked 
input_survey.dropna(inplace=True)

In [ ]:
import importlib
importlib.reload(helper)

# data engineering portion

In [ ]:
#first implementation of pipe for pandas
input_processed = (input_survey.
pipe(spell_chcker,['Comments']).
pipe(text_blob_polarity_semantic_eng,['Comments_spell_checked']).
pipe(time_stamp_cleaner, 'ActivityCompleted')
)

In [ ]:
from classifier import TopicClassifier
tags = ["User Experience", "customer service", "equipment availability", 
"pricing error", 
"incorrect information", 
"login", 
"transaction",
"complexity"]
zero_shot=TopicClassifier(tags) #model init first time is ruff.

In [ ]:
# topic classifier
# input_processed["topic"] = input_processed['Comments_spell_checked'].apply(zero_shot.top_prediction)

# visualiation building

In [ ]:
input_survey = pd.read_csv('../data/cleaned_data.csv')

In [ ]:
# charts and basic information that people might care about ->
nps_best=' '.join(input_survey[input_survey['nps_score'] > 3 ].nlargest(n=50, columns=['Comments_spell_checked_txtblob_polarity','Comments_spell_checked_txtblob_polarity_subjectivity'])['Comments'])
nps_worse=' '.join(input_survey[input_survey['nps_score'] < 3 ].nsmallest(n=50, columns=['Comments_spell_checked_txtblob_polarity','Comments_spell_checked_txtblob_polarity_subjectivity'])['Comments'])
#destribute that text blurb to people about the issues occuring

In [ ]:
sentiment_chart=input_survey.groupby('Sentiment').size()
fig = px.pie(values=sentiment_chart,names=sentiment_chart.index,title='Semantic Count Overall')
fig.show()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
mean_nps_time=input_survey.groupby('ActivityCompleted').mean()
fig = go.Figure(
    [go.Scatter(x=mean_nps_time.index, y=mean_nps_time['nps_score'])],
    layout=go.Layout(
    title=go.layout.Title(text="Average Satisfaction Score")
    )  
)
fig.show()

In [154]:
input_survey.head()

,ActivityCompleted,Status,nps_score,Comments,Sentiment,Device Type,Browser,Comments_spell_checked,Comments_spell_checked_textblob_polarity_score,Comments_spell_checked_txtblob_polarity,Comments_spell_checked_txtblob_polarity_subjectivity,topic,id
0,2022-08-31,Complete,5.0,Zz u CB\nM. L🛖🛖🛖🛖,Neutral,Mobile,Google App,Zz u B M. L🛖🛖🛖🛖,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,transaction,0
1,2022-08-31,Complete,5.0,I am hoping that 1 out of 5\nbeing 1 bad and 5...,Mixed,Desktop,Chrome,I am hoping that 1 out of 5 being 1 bad and 5 ...,"Sentiment(polarity=-0.11666666666666663, subje...",-0.116667,0.755556,incorrect information,1
2,2022-08-31,Complete,3.0,command center is slo processing payment,Negative,Desktop,Chrome,command center is slo processing payment,"Sentiment(polarity=-0.1, subjectivity=0.1)",-0.100000,0.100000,transaction,2
3,2022-08-31,Complete,5.0,Great website easy to use,Positive,Mobile,Safari mobile,Great website easy to use,"Sentiment(polarity=0.6166666666666667, subject...",0.616667,0.791667,User Experience,3
4,2022-08-31,Complete,4.0,"Because nothing is perfect...otherwise a ""5""",Mixed,Desktop,Chrome,"Because nothing is perfect...otherwise a ""5""","Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,User Experience,4


In [152]:
topic_graph=input_survey.groupby(['topic', 'Sentiment']).count()
topic_graph.reset_index(inplace=True)
topic_graph=topic_graph.pivot(index='topic', columns='Sentiment', values='ActivityCompleted')
topic_graph.reset_index(inplace=True)

In [166]:
topic_graph=pd.crosstab(input_survey.topic, input_survey.Sentiment ,normalize = 'index')
topic_graph.reset_index(inplace=True)

In [167]:
import plotly.express as px
fig = px.bar(topic_graph, x="topic", y=["Positive", "Neutral", "Mixed","Negative"], title="Wide-Form Input")
fig.show()

In [ ]:
# subjectivity information how does it compare to the built